# Overview

This notebook includes prototypical code for the operator overview write-up. It does not depend on any exisiting code in DiffEqOperators, but it should be easy to modify the current codebase to achieve the same functionality.

To make things simpler, everything is assumed to use `Float64` datatype. I will also always use the out-of-place convention (i.e. `*` instead of `A_mul_B!`).

The naming of different operators will use the following convention, as in the writeup:

- `L` denotes a (quasi)linear operator.

- `A` denotes an operator that can generally (but not necessarily) be affine.

- `b` denotes the bias of an affine operator

- (Not discussed but I guess we may as well make the call?) `M` for raw matrices (`AbstractMatrix`) and `x`, `y`, `u`, etc for vectors.

In [1]:
import Base: +, -, *, \

abstract type DiffEqOperator end
abstract type DiffEqLinearOperator <: DiffEqOperator end

# 1. Constant case (i.e. no `update_coefficients!`)

## 1.1 Abstract operator interface

Below is a simplified operator interface used in my new interface draft. Basically we want to express lazy addition and multiplication of linear operators naturally using `+` and `*`. The `as_array` interface returns the most suitable (dense/sparse) representation of the underlying operator as an `AbstractMatrix` (or should we treat this as a type conversion and just use `AbstractMatrix`?).

The affine operator is defined in such a way that arithmetic on them and linear operators always yield an `DiffEqAffineOperator`.

In [2]:
# We should improve type stability by including more inferrable fields in the type signature
# But for the sake of this notebook I'll make things simple
struct DiffEqArrayOperator <: DiffEqLinearOperator
    M::AbstractMatrix{Float64}
end

*(L::DiffEqArrayOperator, x::Vector{Float64}) = L.M * x
as_array(L::DiffEqArrayOperator) = L.M

struct DiffEqOperatorCombination <: DiffEqLinearOperator
    cs::Tuple{Vararg{Float64}} # Coefficients
    Ls::Tuple{Vararg{DiffEqLinearOperator}}
end

*(L::DiffEqOperatorCombination, x::Vector{Float64}) = sum(ck * (Lk*x) for (ck,Lk) in zip(L.cs,L.Ls))
as_array(L::DiffEqOperatorCombination) = sum(ck * as_array(Lk) for (ck,Lk) in zip(L.cs,L.Ls))

struct DiffEqOperatorComposition <: DiffEqLinearOperator
    Ls::Tuple{Vararg{DiffEqLinearOperator}}
end

*(L::DiffEqOperatorComposition, x::Vector{Float64}) = foldl((u, Lk) -> Lk*u, x, L.Ls)
as_array(L::DiffEqOperatorComposition) = prod(as_array, reverse(L.Ls))

struct DiffEqAffineOperator <: DiffEqOperator
    L::DiffEqLinearOperator
    b::Vector{Float64}
end

*(A::DiffEqAffineOperator, x::Vector{Float64}) = A.L * x + A.b;

In [3]:
# Operator arithmetic
# Addition
+(L1::DiffEqOperatorCombination, L2::DiffEqOperatorCombination) = DiffEqOperatorCombination(
    (L1.cs...,L2.cs...), (L1.Ls...,L2.Ls...))
+(L1::DiffEqOperatorCombination, L2::DiffEqLinearOperator) = DiffEqOperatorCombination((L1.cs...,1.0), (L1.Ls...,L2))
+(L1::DiffEqLinearOperator, L2::DiffEqOperatorCombination) = L2 + L1
+(L1::DiffEqLinearOperator, L2::DiffEqLinearOperator) = DiffEqOperatorCombination((1.0,1.0), (L1,L2))

+(A1::DiffEqAffineOperator, L2::DiffEqLinearOperator) = DiffEqAffineOperator(A1.L + L2, A1.b)
+(L1::DiffEqLinearOperator, A2::DiffEqAffineOperator) = A2 + L1
+(A1::DiffEqAffineOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(A1.L + A2.L, A1.b + A2.b)

# Scalar multiplication
*(α::Float64, L::DiffEqOperatorCombination) = DiffEqOperatorCombination(α.*L.cs, L.Ls)
*(α::Float64, L::DiffEqLinearOperator) = DiffEqOperatorCombination((α,), (L,))
*(α::Float64, A::DiffEqAffineOperator) = DiffEqAffineOperator(α * A.L, α * A.b)
*(A::DiffEqOperator, α::Float64) = α * A

# Subtraction/unary minus
-(A::DiffEqOperator) = (-1.0) * A
-(A1::DiffEqOperator, A2::DiffEqOperator) = A1 + (-A2)

# Multiplication
# Note the application order
*(L1::DiffEqOperatorComposition, L2::DiffEqOperatorComposition) = DiffEqOperatorComposition((L2.Ls..., L1.Ls...))
*(L1::DiffEqLinearOperator, L2::DiffEqOperatorComposition) = DiffEqOperatorComposition((L2.Ls..., L1))
*(L1::DiffEqOperatorComposition, L2::DiffEqLinearOperator) = DiffEqOperatorComposition((L2, L1.Ls...))
*(L1::DiffEqLinearOperator, L2::DiffEqLinearOperator) = DiffEqOperatorComposition((L2, L1))

*(L1::DiffEqLinearOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(L1 * A2.L, L1 * A2.b)
*(A1::DiffEqAffineOperator, L2::DiffEqLinearOperator) = DiffEqAffineOperator(A1.L * L2, A1.b)
*(A1::DiffEqAffineOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(A1.L * A2.L, A1.L * A2.b + A1.b)

# Right division (i.e. linear solve)
# In the full version we should also include interface to lazy solvers
\(L::DiffEqLinearOperator, y::Vector{Float64}) = as_array(L) \ y
\(A::DiffEqAffineOperator, y::Vector{Float64}) = A.L \ (y - A.b);

## 1.2 Discretization of the differential operator (stencil convolution)

The 2nd-order central difference approximation to $\partial_x^2$ and the 1st-order upwind approximation to $\partial_x$ are included. The general case can be modified from the stencil convolution code in DiffEqOperators.

The upwind operator is implemented differently from DiffEqOperators, where the direction at each point is stored. Here only the left/right operator is constructed and we interpret

$$ \mu(x)\partial_x = \mu^+(x)\partial_x^+ + \mu^-(x)\partial_x^- $$

(Might not be a favorable approach, especially if we wish to extend to multidimensional upwind operators)

In [4]:
struct DiffusionOperator <: DiffEqLinearOperator
    dx::Float64
    m::Int # number of interior points
end

*(L::DiffusionOperator, x::Vector{Float64}) = [x[i] + x[i+2] - 2*x[i+1] for i in 1:L.m] / L.dx^2
as_array(L::DiffusionOperator) = spdiagm((ones(L.m), -2*ones(L.m), ones(L.m)), (0,1,2)) / L.dx^2;

In [5]:
struct DriftOperator <: DiffEqLinearOperator
    dx::Float64
    m::Int # number of interior points
    direction::Bool # true = right, false = left
end

function *(L::DriftOperator, x::Vector{Float64})
    if L.direction # right drift
        [x[i+1] - x[i] for i in 1:L.m] / L.dx
    else # left drift
        [x[i+2] - x[i+1] for i in 1:L.m] / L.dx
    end
end
function as_array(L::DriftOperator)
    if L.direction # right drift
        spdiagm((-ones(L.m), ones(L.m), zeros(L.m)), (0,1,2)) / L.dx
    else # left drift
        spdiagm((zeros(L.m), -ones(L.m), ones(L.m)), (0,1,2)) / L.dx
    end
end;

# 1.3 Boundary extrapolation operator $Q$

Question: is it OK to shorthand "boundary extrapolation operator" as "BEOperator" or simply "BE"?

A generic $Q$ from the generic boundary condition $Bu = b$ can be a bit difficult to implement, but the simple case of Dirichlet/Neumann BC is easy to handle.

It should be easy to modify `AbsorbingBoundaryMap` and `ReflectingBoundaryMap` to incorporate boundaries that are absorbing at one end and reflecting at another.

The non-zero Dirichlet/Neumann Qs do not have their own type. Instead we construct the operator as an affine map, with `AbsorbingBoundaryMap` and `ReflectingBoundaryMap` its linear part.

In [6]:
struct AbsorbingBE <: DiffEqLinearOperator
    m::Int # number of interior points
end

*(Q::AbsorbingBE, x::Vector{Float64}) = [0.0; x; 0.0]
as_array(Q::AbsorbingBE) = sparse([zeros(Q.m)'; eye(Q.m); zeros(Q.m)'])

struct ReflectingBE <: DiffEqLinearOperator
    m::Int # number of interior points
end

*(Q::ReflectingBE, x::Vector{Float64}) = [x[1]; x; x[end]]
as_array(Q::ReflectingBE) = sparse([1.0 zeros(Q.m-1)'; eye(Q.m); zeros(Q.m-1)' 1.0]);

In [7]:
function Dirichlet_BE(m::Int, bl::Float64, br::Float64)
    # y[1] = bl, y[end] = br
    L = AbsorbingBE(m)
    b = [bl; zeros(m); br]
    DiffEqAffineOperator(L, b)
end

function Neumann_BE(m::Int, dx::Float64, bl::Float64, br::Float64)
    # (y[2] - y[1])/dx = bl, (y[end] - y[end-1])/dx = br
    L = ReflectingBE(m)
    b = [-bl*dx; zeros(m); br*dx]
    DiffEqAffineOperator(L, b)
end;

Examples of non-zero BC:

In [8]:
dx = 1.0
u = [1.,2.,3.,4.]
QD = Dirichlet_BE(4, 10.0, 20.0)
QN = Neumann_BE(4, dx, 1.0, 2.0);

In [9]:
QD * u

6-element Array{Float64,1}:
 10.0
  1.0
  2.0
  3.0
  4.0
 20.0

In [10]:
QN * u

6-element Array{Float64,1}:
 0.0
 1.0
 2.0
 3.0
 4.0
 6.0

## 1.4 Constructing operators for the Fokker-Planck equations

The most general case is in section 3.5 of the write-up:

$$ \mathcal{L} = \mu(x)\partial_x + \frac{\sigma(x)^2}{2}\partial_{xx} $$

where the drift term is discretized using upwind operators as described in section 2:

$$ \mu(x)\partial_x = \mu^+(x)\partial_x^+ + \mu^-(x)\partial_x^- $$

The discretized operators are expressed as the composition of an interior stencil convolution operator `L` and boundary extrapolation operator `Q` (they are generally affine). The drift and diffusion coefficients can be expressed as diagonal matrices (wrapped in a `DiffEqArrayOperator`).

The script below can be modified to represent each of the scenarios described in secition 3 of the write-up (except the mixed-BC case).

In [11]:
# The grid
N = 10
dx = 1.0
xs = collect(1:N) * dx # interior nodes

# Discretization of the differential operators
L1p = DriftOperator(dx, N, true)
L1m = DriftOperator(dx, N, false)
L2 = DiffusionOperator(dx, N)

# Boundary operators
Q = AbsorbingBE(N)
# Q = Neumann_BE(N, dx, 1.0, 2.0)

# Coefficients
mu = rand(N)
mup = [mu[i] > 0.0 ? mu[i] : 0.0 for i in 1:N]
mum = [mu[i] < 0.0 ? mu[i] : 0.0 for i in 1:N]
sigma = rand(N) + 1.0

# Construct the final product
Ldrift = DiffEqArrayOperator(Diagonal(mup)) * L1p + DiffEqArrayOperator(Diagonal(mum)) * L1m
Ldiffusion = DiffEqArrayOperator(Diagonal(sigma.^2 / 2)) * L2
A = (Ldrift + Ldiffusion) * Q

DiffEqOperatorComposition((AbsorbingBE(10), DiffEqOperatorCombination((1.0, 1.0, 1.0), (DiffEqOperatorComposition((DriftOperator(1.0, 10, true), DiffEqArrayOperator([0.174104 0.0 … 0.0 0.0; 0.0 0.952114 … 0.0 0.0; … ; 0.0 0.0 … 0.761347 0.0; 0.0 0.0 … 0.0 0.378555]))), DiffEqOperatorComposition((DriftOperator(1.0, 10, false), DiffEqArrayOperator([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]))), DiffEqOperatorComposition((DiffusionOperator(1.0, 10), DiffEqArrayOperator([1.4769 0.0 … 0.0 0.0; 0.0 1.72067 … 0.0 0.0; … ; 0.0 0.0 … 0.689607 0.0; 0.0 0.0 … 0.0 1.54688])))))))

(The standard printout for the composed operator is a bit messy. Should probably implement `Base.show` for the composition types)

In [12]:
# Solve the HJBE (rI - A)u = x
r = 0.05
LHS = DiffEqArrayOperator(r*speye(N)) - A
u = LHS \ xs

10-element Array{Float64,1}:
 19.0869
 35.8929
 43.2801
 46.9296
 46.775 
 44.3825
 40.6502
 34.9724
 23.8643
 13.6993

## 1.5 Alternative: embedding coefficients within the discretization operators

It might be preferrable to use a single type for a differential operator prepended with coefficients, e.g. $\mu(x)\partial_x$ and $\frac{\sigma(x)^2}{2}\partial_x^2$, instead of constructing them as operator compositions. Possible advantages:

- Faster `A_mul_B!`

- Cleaner type signature (also for `update_coefficients!` in the time-dependent case)

- Better handling of upwind operators, especially in >= 1D

In [13]:
struct DiffusionOperatorC{Cs} <: DiffEqLinearOperator
    dx::Float64
    m::Int # number of interior points
    coeffs::Cs # nothing or scalar or vector
end
DiffusionOperatorC(dx, m, coeffs=nothing) = DiffusionOperatorC{typeof(coeffs)}(dx, m, coeffs)

# Handle L*x differently depending on the type of coefficients
*(L::DiffusionOperatorC{Void}, x::Vector{Float64}) = [x[i] + x[i+2] - 2*x[i+1] for i in 1:L.m] / L.dx^2
*(L::DiffusionOperatorC{Float64}, x::Vector{Float64}) = [x[i] + x[i+2] - 2*x[i+1] for i in 1:L.m] * L.coeffs / L.dx^2
*(L::DiffusionOperatorC{Vector{Float64}}, x::Vector{Float64}) = [L.coeffs[i] * (x[i] + x[i+2] - 2*x[i+1]) 
    for i in 1:L.m] / L.dx^2

# Override scalar multiplication
*(α::Float64, L::DiffusionOperatorC{Void}) = DiffusionOperatorC(L.dx, L.m, α)
*(α::Float64, L::DiffusionOperatorC{T}) where {T <: Union{Float64,Vector{Float64}}} = DiffusionOperatorC(
    L.dx, L.m, α*L.coeffs)

as_array(L::DiffusionOperatorC{Void}) = spdiagm((ones(L.m), -2*ones(L.m), ones(L.m)), (0,1,2)) / L.dx^2;
as_array(L::DiffusionOperatorC{Float64}) = spdiagm((ones(L.m), -2*ones(L.m), ones(L.m)), (0,1,2)) * L.coeffs / L.dx^2;
as_array(L::DiffusionOperatorC{Vector{Float64}}) = spdiagm(L.coeffs) * spdiagm(
    (ones(L.m), -2*ones(L.m), ones(L.m)), (0,1,2)) / L.dx^2;

In [14]:
struct DriftOperatorC{Cs} <: DiffEqLinearOperator
    dx::Float64
    m::Int # number of interior points
    coeffs::Cs # scalar or vector
end
DriftOperatorC(dx, m, coeffs=1.0) = DriftOperatorC{typeof(coeffs)}(dx, m, coeffs)

# Handle L*x differently depending on the type of coefficients
function *(L::DriftOperatorC{Float64}, x::Vector{Float64})
    if L.coeffs > 0 # right drift
        [x[i+1] - x[i] for i in 1:L.m] * L.coeffs / L.dx
    else # left drift
        [x[i+2] - x[i+1] for i in 1:L.m] * L.coeffs / L.dx
    end
end
function *(L::DriftOperatorC{Vector{Float64}}, x::Vector{Float64})
    out = zeros(L.m)
    for i in 1:L.m
        c = L.coeffs[i]
        if c > 0 # right drift
            out[i] = c/L.dx * (x[i+1] - x[i])
        else # left drift
            out[i] = c/L.dx * (x[i+2] - x[i+1])
        end
    end
    out
end

# Override scalar multiplication
*(α::Float64, L::DriftOperatorC{T}) where {T <: Union{Float64,Vector{Float64}}} = DriftOperatorC(
    L.dx, L.m, α*L.coeffs)

function as_array(L::DriftOperatorC{Float64})
    if L.coeffs > 0 # right drift
        spdiagm((-ones(L.m), ones(L.m), zeros(L.m)), (0,1,2)) * L.coeffs / L.dx
    else # left drift
        spdiagm((zeros(L.m), -ones(L.m), ones(L.m)), (0,1,2)) * L.coeffs / L.dx
    end
end
function as_array(L::DriftOperatorC{Vector{Float64}})
    out = zeros(L.m, L.m+2)
    for j in 1:L.m
        c = L.coeffs[j]
        if c > 0 # right drift
            out[j,j] = -c/L.dx
            out[j,j+1] = c/L.dx
        else # left drift
            out[j,j+1] = -c/L.dx
            out[j,j+2] = c/L.dx
        end
    end
    out
end;

Test equivalence

In [15]:
N = 10
mu = rand(N)
mup = [x > 0 ? x : 0.0 for x in mu]
mum = [x < 0 ? x : 0.0 for x in mu]
sigma = rand(N)
u = rand(N + 2)

# Diffusion operator
L2 = DiffEqArrayOperator(Diagonal(sigma.^2/2)) * DiffusionOperator(dx, N) # original
L2alt = DiffusionOperatorC(dx, N, sigma.^2/2) # coefficients-embedded
@show L2*u ≈ L2alt * u
@show as_array(L2) ≈ as_array(L2alt)

# Drift Operator
L1 = DiffEqArrayOperator(Diagonal(mup)) * DriftOperator(dx, N, true) + 
    DiffEqArrayOperator(Diagonal(mum)) * DriftOperator(dx, N, false)
L1alt = DriftOperatorC(dx, N, mu)
@show L1*u ≈ L1alt*u
@show as_array(L1) ≈ as_array(L1alt);

L2 * u ≈ L2alt * u = true
as_array(L2) ≈ as_array(L2alt) = true
L1 * u ≈ L1alt * u = true
as_array(L1) ≈ as_array(L1alt) = true
